### Extracted 10 year historical price data for the top 25 SP500 stocks (ranked by MarketCap -- according to Investopedia) using Yahoo Finance API through RapidAPI.

#### columns --> date, date_utc, open, high, low, close, adjclose, volume, symbol 

In [6]:
'''
Python Script that pulls 10 years of historical price data for the top 25 SP500 Stocks

'''

# import libraries
import requests
import json 
from dotenv import load_dotenv
import os, glob 
import pandas as pd

# list of top 25 SP500 stocks 
top_25_sp500_stocks = ['AAPL', 'MSFT', 'AMZN', 'TSLA', 'GOOGL', 
                       'GOOG', 'BRK-B', 'UNH', 'JNJ', 'XOM', 
                       'JPM', 'META', 'V', 'PG', 'NVDA', 'HD',
                       'CVX', 'LLY', 'MA', 'ABBV', 'PFE', 'MRK', 
                       'PEP', 'BAC', 'KO']

# loop through list of stocks and pull data from API 
for stock in top_25_sp500_stocks: 

    url = "https://yahoo-finance15.p.rapidapi.com/api/yahoo/hi/history/" + stock + "/1d"
    
    querystring = {"diffandsplits":"false"}

    load_dotenv()

    X_RapidAPI_Key = os.getenv("X-RapidAPI-Key")

    headers = {
        "X-RapidAPI-Key": X_RapidAPI_Key,
	    "X-RapidAPI-Host": "yahoo-finance15.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    # store API data 
    data = response.json()

    # store stock symbol 
    symbol_df = pd.json_normalize(data['meta'])

    # pull stock details such as open, low, high, close, adjclose, and volume 
    stock_details = []
    for key, val in data['items'].items():
        stock_details.append(val)

    # store stock details 
    stock_details_df = pd.json_normalize(stock_details)

    # add stock symbol into dataframe
    stock_details_df['symbol'] = symbol_df['symbol']
    stock_details_df['symbol'].fillna(stock, inplace=True)
   
    # create csv
    stock_details_df.to_csv('../top25_SP500/data_' + stock + '_historical_prices.csv', index = False)

KeyError: 'items'

In [4]:
# merge top 25 SP500 stock data csv files in to a single csv for data analysis 
path = "../Resources/"

all_files = glob.glob(os.path.join(path, "data_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv( "../merged_top25_sp500_stock_data.csv")

ValueError: No objects to concatenate

In [ ]:
import pandas as pd

# review merged stock data 
merged_top25_sp500_stocks_df = pd.read_csv(
    '../Resources/merged_top25_sp500_stock_data.csv',
    index_col='date', 
    infer_datetime_format=True,
    parse_dates=True
)

# drop un-necessary columns
merged_top25_sp500_stocks_df = merged_top25_sp500_stocks_df.loc[:, ~merged_top25_sp500_stocks_df.columns.str.contains('^Unnamed')]
merged_top25_sp500_stocks_df.drop(['date_utc'], axis=1, inplace=True)

merged_top25_sp500_stocks_df.info()

display(merged_top25_sp500_stocks_df.head())
display(merged_top25_sp500_stocks_df.tail())